In [10]:
import sys
sys.path.append("projects/osu!tools/")
from db import *
osu_osu_file = 'C:\\Users\\Ar\\AppData\\Local\\osu!\\osu!.db'
osu_collection_file = 'C:\\Users\\Ar\\AppData\\Local\\osu!\\collection.db'

In [78]:
with open(osu_collection_file, "rb") as f:
    coll_db = CollectionDB(f)

version: 20190326
no. of collections: 8
collection name: _Attic
no. of beatmaps: 235
beatmap#1 MD5: b0140f0a92d5aee17239584b3eb20352
beatmap#2 MD5: a1fe604eeed945a2557fbdc4c9b624ba
beatmap#3 MD5: 0fb944a47b004cc03ceac0774ff78987
beatmap#4 MD5: d9a6ee05d23b026f3f0f627ea976e601
beatmap#5 MD5: af8a3c450453e4dad2a92b287dccad44
beatmap#6 MD5: cb5477d1fd28846e57f71e8869fca582
beatmap#7 MD5: 1e93b8c7b5bb0e270748ff692ef2534a
beatmap#8 MD5: d874ca280e95883296707dc871e17722
beatmap#9 MD5: f7570b5b21277dd0993ac05655e9cce5
beatmap#10 MD5: 8c354852054f66bc56511eaa8467b948
beatmap#11 MD5: 2a529238e35b0e706f8c8c8cdcb694cb
beatmap#12 MD5: 6b1dde076d67f7d52f2cc1e7b99204da
beatmap#13 MD5: a617d173d2042a13bd15cccaf0d88554
beatmap#14 MD5: 1af605f6a3dd2b950526928e4484e19f
beatmap#15 MD5: fb7e16e28e4fcb25fbbed2a7d1e6f4b3
beatmap#16 MD5: 18a22d09ff38b9a30f713ef295ea1e43
beatmap#17 MD5: 9801fd196170a5671d95a125ae1cc4fa
beatmap#18 MD5: 94b8a73448a9ed949824e7bc41b6bf90
beatmap#19 MD5: 65edfd8b47ad8c9408f73e2c89

In [70]:
OSU_GAME_TYPES = ['standard', 'taiko', 'ctb', 'mania']
OSU_BEATMAP_RANKED_STATUS = ['unknown', 'unsubmitted', 'pending/wip/graveyard', 'unused','ranked', 'approved', 'qualified', 'loved']

beatmap_list = []

with open(osu_osu_file, "rb") as f:
    
    osu_version, folder_count = read_int(f), read_int(f)
    print('osu! version:', osu_version)
    print('Folder Count:', folder_count)
    
    account_unlocked, date_account_unlocked = read_bool(f), read_datetime(f)
    print('AccountUnlocked:', account_unlocked)
    print('Date the account will be unlocked:', date_account_unlocked)
    
    player_name = read_string(f)
    print('Player name:', player_name)
    
    beatmap_count = read_int(f)
    print('Number of beatmaps:', beatmap_count)
    
#     beatmap_count = 5
    for c in range(beatmap_count):
    
        entry_size = read_int(f)
        artist_name, artist_name_u = read_string(f), read_string(f)
        song_title, song_title_u = read_string(f), read_string(f)
        creator_name = read_string(f)
        difficulty = read_string(f)
        audio_file_name = read_string(f)
        beatmap_hash = read_string(f)
        osu_filename = read_string(f)
        ranked_status = read_byte(f)
        hitcircle_count, slider_count, spinner_count = read_short(f), read_short(f), read_short(f)
        last_mod_time = read_datetime(f)
        approach_rate, circle_size, hp_drain = read_single(f), read_single(f), read_single(f)
        overall_difficulty = read_single(f)
        slider_velocity = read_double(f)
        star_ratings = {}
        for game_type in OSU_GAME_TYPES:
            pair_count = read_int(f)
            
            mod_stars = []
            for _ in range(pair_count):
                read_byte(f)  # 0x08
                mod_comb = read_int(f)
                read_byte(f)  # 0x0d
                stars = read_double(f)
                mod_stars.append((mod_comb, stars))
                
            star_ratings[game_type] = mod_stars
        
        drain_time, total_time, preview_time = read_int(f), read_int(f), read_int(f)
        timing_point_count = read_int(f)
        timing_points = []
        for _ in range(timing_point_count):
            bpm, offset, inherited = read_double(f), read_double(f), not read_bool(f)
            timing_points.append((bpm, offset, inherited))
        
        beatmap_id, beatmap_set_id, thread_id = read_int(f), read_int(f), read_int(f)
        grades = {game_type: read_byte(f) for game_type in OSU_GAME_TYPES}
        local_beatmap_offset = read_short(f)
        stack_leniency = read_single(f)
        gameplay_mode = read_byte(f)
        song_source, song_tags = read_string(f), read_string(f)
        online_offset = read_short(f)
        song_font = read_string(f)
        beatmap_unplayed = read_bool(f)
        beatmap_last_played = read_datetime(f)
        beatmap_osz2 = read_bool(f)
        beatmap_folder = read_string(f)
        beatmap_last_checked = read_datetime(f)#long(f)
        ignore_sound, ignore_skin, disable_storyboard, disable_video, visual_override = \
            read_bool(f), read_bool(f), read_bool(f), read_bool(f), read_bool(f)
        last_modified = read_int(f)
        mania_scroll_speed = read_byte(f)
        
        beatmap_list.append((beatmap_hash, song_title, difficulty, artist_name))
        
    print('Length of beatmap_list:', len(beatmap_list))
#     print('beatmap_list:', beatmap_list)

# collect hashs from all beatmaps
beatmap_hashs = [x[0] for x in beatmap_list]

osu! version: 20190326
Folder Count: 331
AccountUnlocked: True
Date the account will be unlocked: 0001-01-01 00:00:00
Player name: arenaaz
Number of beatmaps: 1806
Length of beatmap_list: 1806


In [79]:
# collect hashs from all collections
coll_hashs = []
for name, coll in coll_db.collections.items():
    print('collection name:', (name, coll), len(coll.beatmap_md5s))
    coll_hashs.extend(coll.beatmap_md5s)
print('total number of hashes in collections:', len(coll_hashs))

collection name: ('_Attic', <db.Collection object at 0x0000014F9FF67A20>) 235
collection name: ('_Long', <db.Collection object at 0x0000014F9FF42128>) 112
collection name: ('_Pending', <db.Collection object at 0x0000014F9FF67EF0>) 158
collection name: ('_Skill - Jumps', <db.Collection object at 0x0000014F9FF67C88>) 34
collection name: ('_Skill - Streams', <db.Collection object at 0x0000014F9FF67198>) 5
collection name: ('Can do better', <db.Collection object at 0x0000014F9FF67DD8>) 70
collection name: ('Nice', <db.Collection object at 0x0000014F9FF67208>) 792
collection name: ('Practice', <db.Collection object at 0x0000014F9FCA3160>) 1201
total number of hashes in collections: 2607


In [81]:
# find beatmap hashs not in any collections
hashs_not_in_coll = list(set(beatmap_hashs) - set(coll_hashs))
print('number of beatmap hashs not in any collections:', len(hashs_not_in_coll))

# for bm in beatmap_list:
#     print('bm[0]:', bm[0])
#     if bm[0] == hashs_not_in_coll[0]:
#         print('bingo!')

# reveal name of beatmap not in any collections
beatmapss_not_in_coll = [bm for bm in beatmap_list if bm[0] in hashs_not_in_coll]
beatmapss_not_in_coll

number of beatmap hashs not in any collections: 78


[('', '', '', ''),
 ('85fc5c7b25621f773c58bc98398bf3b6',
  'INNOCENCE (TV Size)',
  'Easy',
  'Aoi Eir'),
 ('8a037454998b3bb93439464f32bfd214',
  'INNOCENCE (TV Size)',
  "NoHitter's Normal",
  'Aoi Eir'),
 ('bc61e426bbedb8743c01bafcf3b2be96',
  'SHIORI (TV Size)',
  "Alphabet's Extra",
  'ClariS'),
 ('fbc064ee1e7d34051ce049efc105b785', 'SHIORI (TV Size)', 'Easy', 'ClariS'),
 ('5f184adc41f46dadf94615de5ec23628',
  'SHIORI (TV Size)',
  'Everlasting',
  'ClariS'),
 ('cde071c842f3e954a99e67a37de5c273', 'SHIORI (TV Size)', 'Hard', 'ClariS'),
 ('350610c7daf2dc73caee5cfd46bac0c6',
  'SHIORI (TV Size)',
  "Lasse's Expert",
  'ClariS'),
 ('0a2651f8f234d58f2de42f7c020bd5c9',
  'SHIORI (TV Size)',
  "Lukas' Insane",
  'ClariS'),
 ('e1994024030939baedd9ec732504bf8b',
  'SHIORI (TV Size)',
  "Nevo's Extra",
  'ClariS'),
 ('7803204836fad1de4c85ac15bf635c78', 'SHIORI (TV Size)', 'Normal', 'ClariS'),
 ('fd0d43f29e04c3f15d0742cb85f4484f',
  'SHIORI (TV Size)',
  "Xexxar's Epilogue",
  'ClariS'),
 ('a